<a href="https://colab.research.google.com/github/viniciusrpb/cloud_image_classification/blob/main/cloud_classification_swimcat_BEiT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloud Image Classification using Vision Transformers (Beit)

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#!cp -r "/content/drive/My Drive/img_satelite/classificacao/SWIMCAT/train" "training"
#!cp -r "/content/drive/My Drive/img_satelite/classificacao/SWIMCAT/val" "validation"
#!cp -r "/content/drive/My Drive/img_satelite/classificacao/SWIMCAT/test" "testing"

In [3]:
#!pip install pytorch pytorch torchvision
#!pip install timm==0.3.2
#!pip install datasets transformers
#!pip install transformers pytorch-lightning --quiet
#!sudo apt -qq install git-lfs

In [31]:
from datasets import load_dataset
import tensorflow as tf
import torchvision
from torchvision.transforms import ToTensor
import math
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, UnidentifiedImageError
from pathlib import Path
import torch
import glob
import pytorch_lightning as pl
from huggingface_hub import HfApi, Repository
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchmetrics import Accuracy
from transformers import DeiTForImageClassification,BeitForImageClassification,BeitFeatureExtractor,DeiTFeatureExtractor,BeitFeatureExtractor
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix,classification_report

In [5]:
path_train = 'training'
path_validation = 'validation'
path_test = 'testing'

Define the image generator objects

In [6]:
train_ds = torchvision.datasets.ImageFolder(path_train, transform=ToTensor())
valid_ds = torchvision.datasets.ImageFolder(path_validation, transform=ToTensor())
test_ds = torchvision.datasets.ImageFolder(path_test, transform=ToTensor())

In [7]:
train_ds.classes

['A-sky', 'B-pattern', 'C-thick-dark', 'D-thick-white', 'E-veil']

In [8]:
def fn_collator(batch):
    encodings = feature_extractor([x[0] for x in batch], return_tensors='pt')
    encodings['labels'] = torch.tensor([x[1] for x in batch], dtype=torch.long)
    return encodings 

Pega os códigos das classes do dataset

In [9]:
dic_label2id = {}
dic_id2label = {}
for i, class_name in enumerate(train_ds.classes):
  dic_label2id[class_name] = str(i)
  dic_id2label[str(i)] = class_name

Allocate objects for loading the data using the DataGenerator

In [10]:
from transformers import BeitForImageClassification,BeitFeatureExtractor
model_name_or_path = 'microsoft/beit-base-patch16-224'
feature_extractor = BeitFeatureExtractor.from_pretrained(model_name_or_path)

#model = FlaxBeitForImageClassification.from_pretrained("microsoft/beit-base-patch16-224")
model = BeitForImageClassification.from_pretrained("microsoft/beit-base-patch16-224")

#model = BeitForImageClassification.from_pretrained(
#    model_name_or_path,#
#    num_labels=len(train_ds.classes),
#    id2label=dic_id2label,
#    label2id=dic_label2id)

/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [23]:
import numpy as np
from datasets import load_metric

acc = load_metric("accuracy")
f1score = load_metric("f1",average='micro')
precision = load_metric("precision")
recall = load_metric("recall")

def compute_metrics(p):

    results1 = acc.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)
    results2 = f1score.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids,average='micro')
    results3 = precision.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids,average='micro')
    results4 = recall.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids,average='micro')

    print(results1)

    return {
        "precision": results3,
        "recall": results4,
        "f1": results2,
        "accuracy": results1,
    }

In [24]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./beit-base-clouds",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=2,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=5e-5,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

PyTorch: setting up devices


In [25]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=fn_collator,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=feature_extractor,
)

Using amp half precision backend


In [26]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 546
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 70


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./beit-base-clouds
Configuration saved in ./beit-base-clouds/config.json
Model weights saved in ./beit-base-clouds/pytorch_model.bin
Feature extractor saved in ./beit-base-clouds/preprocessor_config.json


***** train metrics *****
  epoch                    =        2.0
  total_flos               = 79480932GF
  train_loss               =     0.1015
  train_runtime            = 0:02:27.05
  train_samples_per_second =      7.426
  train_steps_per_second   =      0.476


In [28]:
metrics = trainer.evaluate(test_ds)
trainer.log_metrics("eval", metrics['precision'])
#trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 162
  Batch size = 8


Trainer is attempting to log a value of "{'precision': 1.0}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 1.0}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 1.0}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 1.0}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'accuracy': 1.0}


KeyError: ignored

In [29]:
metrics

{'epoch': 2.0,
 'eval_accuracy': {'accuracy': 1.0},
 'eval_f1': {'f1': 1.0},
 'eval_loss': 0.0016945298993960023,
 'eval_precision': {'precision': 1.0},
 'eval_recall': {'recall': 1.0},
 'eval_runtime': 9.1443,
 'eval_samples_per_second': 17.716,
 'eval_steps_per_second': 2.297}

In [47]:
y_true = test_ds.targets
y_pred = trainer.predict(test_ds)

cf_matrix = confusion_matrix(np.array(y_true), np.array(y_pred[1]))

***** Running Prediction *****
  Num examples = 162
  Batch size = 8


{'accuracy': 1.0}


In [48]:
print(classification_report(np.array(y_true), np.array(y_pred[1]), target_names=test_ds.classes))

               precision    recall  f1-score   support

        A-sky       1.00      1.00      1.00        46
    B-pattern       1.00      1.00      1.00        19
 C-thick-dark       1.00      1.00      1.00        51
D-thick-white       1.00      1.00      1.00        28
       E-veil       1.00      1.00      1.00        18

     accuracy                           1.00       162
    macro avg       1.00      1.00      1.00       162
 weighted avg       1.00      1.00      1.00       162

